In [3]:
import pymysql
import pandas as pd
import numpy as np
import datetime
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn import preprocessing
from sklearn.utils import shuffle
from keras import layers
from keras.models import Sequential
from keras import optimizers
from keras.utils import plot_model
import matplotlib.pyplot as plt


import sys
sys.path.append(r'D:\jupyter files\waiting_time_project\my_tools')
import tools_for_os.for_df as ml_df

In [4]:
# 把字符串转成datetime
def str_to_Datetime(st):
    dt = datetime.datetime.strptime(st, "%Y-%m-%d %H:%M:%S")
    return dt

# 把datetime 转成字符串
def Datetime_to_str(d):
    str_date = d.strftime('%Y-%m-%d %H:%M:%S')
    return str_date

# 计算时间差
def cal_time_inv(before, now):
    before = str_to_Datetime(before)
    now = str_to_Datetime(now)
    if now >= before:
        inv = (now - before).seconds
        return inv
    else:
        return -1

=========================================================

#### 这一部分的数据与处理和治疗时长预测的预处理相同

都需要将同一个appointment 里面的特征合并，同时将分类变量进行编码

In [5]:
data_path = 'D:\\jupyter files\\data_waiting_time_project\\preprocess_data\\'

data_part1 = pd.read_csv(data_path + 'data_part1.csv', index_col = 0)
data_part2 = pd.read_csv(data_path + 'data_part2.csv', index_col = 0)

DATA = pd.merge(data_part1, data_part2, on = ['PatientSerNum', 'date'], how = 'inner')
DATA.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'FractionNumber'], inplace = True)
print(f'The shape of data is {DATA.shape}')

The shape of data is (1788434, 35)


In [6]:
feature_num = ['Scheduled_duration', 'Actual_duration',
               'age', 'TreatmentTime_total', 'ImagesTaken_total',
               'MU_total', 'MUCoeff_total', 'Interval_scheduled']

# RadiationId
feature_cate = ['dxt_AliasName', 'Sex', 'AliasSerNum',
                'month', 'week', 'hour', 'DoctorSerNum', 
                'TreatmentOrientation', 'FractionNumber',
                'UserName', 'CourseId', 'ResourceSerNum']

In [7]:
# 因为需要进行onehot encoding，所以在拼接数据之前，先进行数据格式的处理
for col in feature_cate:
    try:
        data_part1[col].fillna('Unknown', inplace = True)
        data_part1[col] = data_part1[col].astype(str)
    except:
        pass
    
    try:
        data_part2[col].fillna('Unknown', inplace = True)
        data_part2[col] = data_part2[col].astype(str)
    except:
        pass

for col in feature_num:
    try:
        data_part1.fillna(0, inplace = True)
    except:
        pass
    try:
        data_part2.fillna(0, inplace = True)
    except:
        pass
    
# data_num = log1p(data_num)

In [8]:
# # one hot encoder
# label_encoder_dxt_AliasName = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_dxt_AliasName.fit(data_part1.dxt_AliasName.values.reshape(-1, 1))

# label_encoder_Sex = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_Sex.fit(data_part1.Sex.values.reshape(-1, 1))

# label_encoder_AliasSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_AliasSerNum.fit(data_part1.AliasSerNum.values.reshape(-1, 1))

# label_encoder_month = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_month.fit(data_part1.month.values.reshape(-1, 1))

# label_encoder_week = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_week.fit(data_part1.week.values.reshape(-1, 1))

# label_encoder_hour = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_hour.fit(data_part1.hour.values.reshape(-1, 1))

# label_encoder_DoctorSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_DoctorSerNum.fit(data_part1.DoctorSerNum.values.reshape(-1, 1))

# label_encoder_TreatmentOrientation = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_TreatmentOrientation.fit(data_part1.TreatmentOrientation.values.reshape(-1, 1))

# label_encoder_FractionNumber = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_FractionNumber.fit(data_part2.FractionNumber.values.reshape(-1, 1))

# label_encoder_UserName = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_UserName.fit(data_part2.UserName.values.reshape(-1, 1))

# label_encoder_CourseId = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_CourseId.fit(data_part2.CourseId.values.reshape(-1, 1))

# label_encoder_ResourceSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_ResourceSerNum.fit(data_part2.ResourceSerNum.values.reshape(-1, 1))

In [9]:
# def get_list(x):
#     x = list(x)
#     if len(x) == 1:
#         return x[0]
#     else:
#         return x

In [10]:
# new_appt = pd.DataFrame({})

# print('Start cateorical features')
# print('\nStart PatientSerNum')
# new_appt['PatientSerNum'] = data_part1.groupby('AppointmentSerNum').PatientSerNum.apply(set)
# new_appt['PatientSerNum'] = new_appt['PatientSerNum'].apply(lambda x: get_list(x))

# print('Start Sex')
# new_appt['Sex'] = data_part1.groupby('AppointmentSerNum').Sex.apply(set)
# new_appt['Sex'] = new_appt['Sex'].apply(lambda x: get_list(x))

# print('Start DoctorSerNum')
# new_appt['DoctorSerNum'] = data_part1.groupby('AppointmentSerNum').DoctorSerNum.apply(set)
# new_appt['DoctorSerNum'] = new_appt['DoctorSerNum'].apply(lambda x: get_list(x))

# print('Start date')
# new_appt['date'] = data_part1.groupby('AppointmentSerNum').date.apply(set)
# new_appt['date'] = new_appt['date'].apply(lambda x: get_list(x))

# print('Start ScheduledStartTime')
# new_appt['ScheduledStartTime'] = data_part1.groupby('AppointmentSerNum').ScheduledStartTime.apply(set)
# new_appt['ScheduledStartTime'] = new_appt['ScheduledStartTime'].apply(lambda x: get_list(x))

# print('Start ScheduledEndTime')
# new_appt['ScheduledEndTime'] = data_part1.groupby('AppointmentSerNum').ScheduledEndTime.apply(set)
# new_appt['ScheduledEndTime'] = new_appt['ScheduledEndTime'].apply(lambda x: get_list(x))

# print('Start ActualStartDate')
# new_appt['ActualStartDate'] = data_part1.groupby('AppointmentSerNum').ActualStartDate.apply(set)
# new_appt['ActualStartDate'] = new_appt['ActualStartDate'].apply(lambda x: get_list(x))

# print('Start ActualEndDate')
# new_appt['ActualEndDate'] = data_part1.groupby('AppointmentSerNum').ActualEndDate.apply(set)
# new_appt['ActualEndDate'] = new_appt['ActualEndDate'].apply(lambda x: get_list(x))

# print('Start dxt_AliasName')
# new_appt['dxt_AliasName'] = data_part1.groupby('AppointmentSerNum').dxt_AliasName.apply(set)
# new_appt['dxt_AliasName'] = new_appt['dxt_AliasName'].apply(lambda x: get_list(x))

# print('Start AliasSerNum')
# new_appt['AliasSerNum'] = data_part1.groupby('AppointmentSerNum').AliasSerNum.apply(set)
# new_appt['AliasSerNum'] = new_appt['AliasSerNum'].apply(lambda x: get_list(x))

# print('Start CourseSerNum')
# new_appt['CourseSerNum'] = data_part1.groupby('AppointmentSerNum').CourseSerNum.apply(set)
# new_appt['CourseSerNum'] = new_appt['CourseSerNum'].apply(lambda x: get_list(x))

# print('Start PlanSerNum')
# new_appt['PlanSerNum'] = data_part1.groupby('AppointmentSerNum').PlanSerNum.apply(set)
# new_appt['PlanSerNum'] = new_appt['PlanSerNum'].apply(lambda x: get_list(x))

# print('Start TreatmentOrientation')
# new_appt['TreatmentOrientation'] = data_part1.groupby('AppointmentSerNum').TreatmentOrientation.apply(set)
# new_appt['TreatmentOrientation'] = new_appt['TreatmentOrientation'].apply(lambda x: get_list(x))

# print('Start month')
# new_appt['month'] = data_part1.groupby('AppointmentSerNum').month.apply(set)
# new_appt['month'] = new_appt['month'].apply(lambda x: get_list(x))

# print('Start week')
# new_appt['week'] = data_part1.groupby('AppointmentSerNum').week.apply(set)
# new_appt['week'] = new_appt['week'].apply(lambda x: get_list(x))

# print('Start hour')
# new_appt['hour'] = data_part1.groupby('AppointmentSerNum').hour.apply(set)
# new_appt['hour'] = new_appt['hour'].apply(lambda x: get_list(x))

# print('Start AppointmentSerNum')
# new_appt['AppointmentSerNum'] = new_appt.index.tolist()


# print('Start numberical features')
# print('\nStart age')
# new_appt['age'] = data_part1.groupby('AppointmentSerNum').age.mean()

# print('Start Scheduled_duration')
# new_appt['Scheduled_duration'] = data_part1.groupby('AppointmentSerNum').Scheduled_duration.mean()

# print('Start Actual_duration')
# new_appt['Actual_duration'] = data_part1.groupby('AppointmentSerNum').Actual_duration.mean()

In [ ]:
# new_treat = pd.DataFrame({})

# print('Start categorical features')
# print('\nStart FractionNumber')
# new_treat['FractionNumber'] = data_part2.groupby(['PatientSerNum', 'date']).FractionNumber.apply(set)
# new_treat['FractionNumber'] = new_treat['FractionNumber'].apply(lambda x: get_list(x))

# print('Start UserName')
# new_treat['UserName'] = data_part2.groupby(['PatientSerNum', 'date']).UserName.apply(set)
# new_treat['UserName'] = new_treat['UserName'].apply(lambda x: get_list(x))

# print('Start RadiationSerNum')
# new_treat['RadiationSerNum'] = data_part2.groupby(['PatientSerNum', 'date']).RadiationSerNum.apply(set)
# new_treat['RadiationSerNum'] = new_treat['RadiationSerNum'].apply(lambda x: get_list(x))

# print('Start RadiationId')
# new_treat['RadiationId'] = data_part2.groupby(['PatientSerNum', 'date']).RadiationId.apply(set)
# new_treat['RadiationId'] = new_treat['RadiationId'].apply(lambda x: get_list(x))

# # print('Start ResourceSerNum')
# # new_treat['ResourceSerNum'] = data_part2.groupby(['PatientSerNum', 'date']).ResourceSerNum.apply(set)
# # new_treat['ResourceSerNum'] = new_treat['ResourceSerNum'].apply(lambda x: get_list(x))

# print('Start CourseId')
# new_treat['CourseId'] = data_part2.groupby(['PatientSerNum', 'date']).CourseId.apply(set)
# new_treat['CourseId'] = new_treat['CourseId'].apply(lambda x: get_list(x))

# print('Start PatientSerNum')
# new_treat['PatientSerNum'] = new_treat.index.get_level_values(level = 0).tolist()

# print('Start date')
# new_treat['date'] = new_treat.index.get_level_values(level = 1).tolist()


# print('Start numberical features')
# print('\nStart ImagesTaken_total')
# new_treat['ImagesTaken_total'] = data_part2.groupby(['PatientSerNum', 'date']).ImagesTaken.sum()

# print('Start MU_total')
# new_treat['MU_total'] = data_part2.groupby(['PatientSerNum', 'date']).MU.sum()

# print('Start MUCoeff_total')
# new_treat['MUCoeff_total'] = data_part2.groupby(['PatientSerNum', 'date']).MUCoeff.sum()

# print('Start TreatmentTime_total')
# new_treat['TreatmentTime_total'] = data_part2.groupby(['PatientSerNum', 'date']).TreatmentTime.sum()

In [ ]:
# new_treat.drop('ResourceSerNum', axis = 1, inplace = True)

In [ ]:
new_appt.reset_index(drop = True, inplace = True)
new_treat.reset_index(drop = True, inplace = True)

print(f'The shape of new_appt is {new_appt.shape}')
print(f'The shape of new_treat is {new_treat.shape}')

DATA = pd.merge(new_appt, new_treat, on = ['PatientSerNum', 'date'], how = 'inner')
DATA.sort_values(by = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime'], inplace = True)
print(f'The shape of data is {DATA.shape}')

In [ ]:
DATA = pd.merge(DATA, data_re_appt[['AppointmentSerNum', 'ResourceSerNum']], on = 'AppointmentSerNum', how = 'inner')
print(f'The shape of data is {DATA.shape}')

In [ ]:
print(f'The shape of data is {DATA.shape}')
DATA_ = DATA[(DATA.Actual_duration >= 10) &
             (DATA.Actual_duration <= 60)]
print(f'The shape of data is {DATA_.shape}')

=========================================================

In [11]:
sql_re = """SELECT * FROM resource"""
data_re_ = ml_df.get_df_from_sql(sql_re)
print(f'The shape of data_resource {data_re_.shape}')
sql_re_appt = """SELECT * FROM resourceappointment"""
data_re_appt_ = ml_df.get_df_from_sql(sql_re_appt)
print(f'The shape of data_re_appt {data_re_appt_.shape}')
sql_appt = """SELECT * FROM appointment"""
data_appt_ = ml_df.get_df_from_sql(sql_appt)
print(f'The shape of data_appt {data_appt_.shape}')
sql_pl = """SELECT * FROM patientlocation"""
data_pl_ = ml_df.get_df_from_sql(sql_pl)
print(f'The shape of data_pl {data_pl_.shape}')
sql_plmh = """SELECT * FROM patientlocationmh"""
data_plmh_ = ml_df.get_df_from_sql(sql_plmh)
print(f'The shape of data_plmh {data_plmh_.shape}')

The shape of data_resource (477, 6)
The shape of data_re_appt (475920, 7)
The shape of data_appt (1074478, 17)
The shape of data_pl (206855, 9)
The shape of data_plmh (2333039, 10)


In [12]:
try:
    print('Drop columns')
    data_appt_.drop(columns = ['LastUpdated'], inplace = True)
    data_re_.drop(columns = ['LastUpdated'], inplace = True)
    data_re_appt_.drop('LastUpdated', axis = 1, inplace = True)
    data_pl_.drop(columns = ['LastUpdated'], inplace = True)
    data_plmh_.drop(columns = ['LastUpdated'], inplace = True)
except:
    print('Finish droppping columns')

Drop columns


In [13]:
data_re = data_re_.copy()
data_re_appt = data_re_appt_.copy()
data_appt = data_appt_.copy()
data_pl = data_pl_.copy()
data_plmh = data_plmh_.copy()


print(f'The shape of data_appt {data_appt.shape}')
data_appt = data_appt[
    (data_appt.ActualStartDate != datetime.datetime(1970, 1, 1, 0, 0)) &
    (data_appt.ActualEndDate != datetime.datetime(1970, 1, 1, 0, 0)) &
    (data_appt.ActualStartDate != '0000-00-00 00:00:00')]
print(f'The shape of data_appt {data_appt.shape}')

# 已经检查过，一个Appointment 里面只有一个AliasSerNum
# 一个AliasSerNum 对应多个ResourceSerNum
# 一个ResourceSerNum 对应多个AppointmentSerNum
print('Merge data_appt and data_re_appt')
appt_re_appt = pd.merge(data_appt, data_re_appt, on = ['AppointmentSerNum', 'AliasSerNum'], how = 'inner')
print(f'The shape of appt_re_appt {appt_re_appt.shape}')
appt_re_appt.drop('AliasSerNum', axis = 1, inplace = True)

print('\nMerge data_re')
appt_loc = pd.merge(data_re, appt_re_appt, on = ['ResourceSerNum'], how = 'inner')
print(f'The shape of appt_loc {appt_loc.shape}')

print('\nMerge data_pl')
data_pl.drop('AliasSerNum', axis = 1, inplace = True)
appt_loc_pl = pd.merge(data_pl, appt_loc, on = ['AppointmentSerNum'], how = 'right')
print(f'The shape of appt_loc_pl {appt_loc_pl.shape}')

print('\nGet Treat_Date')
appt_loc_pl['Treat_Date'] = appt_loc_pl.apply(lambda x: x.ActualStartDate.strftime("%Y-%m-%d"), axis = 1)

appt_loc_pl = appt_loc_pl[[
    'AppointmentSerNum', 'PatientSerNum', 'AliasExpressionSerNum',
    'ScheduledStartTime', 'ScheduledEndTime',
    'ActualStartDate', 'ActualEndDate',
    'ResourceSerNum']]
print(f'The shape of room_infomation {appt_loc_pl.shape}')

The shape of data_appt (1074478, 16)
The shape of data_appt (370299, 16)
Merge data_appt and data_re_appt
The shape of appt_re_appt (165512, 20)

Merge data_re
The shape of appt_loc (165512, 23)

Merge data_pl
The shape of appt_loc_pl (165512, 29)

Get Treat_Date
The shape of room_infomation (165512, 8)


In [15]:
appt_loc_pl.head()

,AppointmentSerNum,PatientSerNum,AliasExpressionSerNum,ScheduledStartTime,ResourceSerNum
0,849926,38996,1505,2015-10-19 14:00:00,176
1,849927,38996,1505,2015-10-20 13:45:00,177
2,849928,38996,1505,2015-10-21 15:30:00,176
3,849929,38996,1505,2015-10-22 14:00:00,176
4,849930,38996,1505,2015-10-23 13:00:00,176


In [14]:
appt_loc_pl = appt_loc_pl[[
    'AppointmentSerNum', 'PatientSerNum', 'AliasExpressionSerNum', 'ScheduledStartTime', 'ResourceSerNum']]
appt_loc_pl['ScheduledStartTime'] = appt_loc_pl['ScheduledStartTime'].apply(lambda x: Datetime_to_str(x))

room_info = pd.merge(appt_loc_pl, DATA_, 
                     on = ['PatientSerNum', 'AppointmentSerNum', 'ScheduledStartTime', 'ResourceSerNum'], how = 'inner')
print(f'The shape of room_infomation {room_info.shape}')

NameError: name 'DATA_' is not defined

In [ ]:
# # one hot encoder
# label_encoder_AliasExpressionSerNum = preprocessing.OneHotEncoder(handle_unknown = 'ignore', sparse = False)
# label_encoder_AliasExpressionSerNum.fit(room_info.AliasExpressionSerNum.values.reshape(-1, 1))

In [ ]:
room_info_grouped = room_info.groupby(['ResourceSerNum', 'date'])

room_count = room_info_grouped.count()
room_seq = room_count[room_count.PatientSerNum > 1].index.tolist()
room_one = room_count[room_count.PatientSerNum == 1].index.tolist()
print(f'room_seq {len(room_seq)}')
print(f'room_one {len(room_one)}')

In [ ]:
def generate_room_seq(room):
    room.sort_values(by = 'ActualStartDate', inplace = True)

    room['pre_ActualEndDate'] = room.ActualEndDate.shift(periods = 1, fill_value = room.ActualStartDate.iloc[0])
    room['Transfer_duration'] = room.apply(lambda x: cal_time_inv(x.pre_ActualEndDate,x.ActualStartDate), axis = 1)
    room['Interval_number'] = room.apply(lambda x: int(x.Transfer_duration / 60 / 15), axis = 1)
    
    label_d = room.Transfer_duration.iloc[-1]
    label_n = room.Interval_number.iloc[-1]
    
    room.Actual_duration.iloc[-1] = 0
    room.Transfer_duration.iloc[-1] = 0
    
#     room.Transfer_duration.iloc[-1] = cal_time_inv(room.ActualEndDate.iloc[-2], room.ScheduledStartTime.iloc[-1])
    
    room.Interval_number.iloc[-1] = int(room.Transfer_duration.iloc[-1] /60 / 15)
    return room, label_d, label_n

In [ ]:
# ['AppointmentSerNum', 'PatientSerNum', 
#  'ScheduledStartTime', 'ResourceSerNum', 
#  'date', 'ScheduledEndTime',
#  'ActualStartDate', 'ActualEndDate', 
#  'CourseSerNum', 'PlanSerNum',
#  'RadiationSerNum', 'RadiationId',
#  'pre_ActualEndDate']


# RadiationId
feature_cate = ['dxt_AliasName', 'Sex', 'AliasSerNum',
                'month', 'week', 'hour', 'DoctorSerNum', 
                'TreatmentOrientation', 'FractionNumber',
                'UserName', 'CourseId', 'ResourceSerNum', 
                'AliasExpressionSerNum']

feature_num = ['Scheduled_duration', 'Actual_duration',
               'age', 'TreatmentTime_total', 'ImagesTaken_total',
               'MU_total', 'MUCoeff_total',
               'Transfer_duration', 'Interval_number']

In [ ]:
def generate_sample(room, feature_num = feature_num):
    
    # 对分类变量进行one-hot encoding处理
    encode_cate = pd.DataFrame({})
    
    # 这个地方将Sex 作为序列的一部分，并不是在最后的隐藏层进行合并
    encode_cate['Sex'] = room['Sex'].apply(
        lambda x: sum(label_encoder_Sex.transform(np.array(x).reshape(-1,1))))
    train_x = np.vstack(encode_cate.Sex.tolist())

    encode_cate['dxt_AliasName'] = room['dxt_AliasName'].apply(
        lambda x: sum(label_encoder_dxt_AliasName.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.dxt_AliasName.tolist())))

    encode_cate['AliasSerNum'] = room['AliasSerNum'].apply(
        lambda x: sum(label_encoder_AliasSerNum.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.AliasSerNum.tolist())))

    encode_cate['month'] = room['month'].apply(
        lambda x: sum(label_encoder_month.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.month.tolist())))

    encode_cate['week'] = room['week'].apply(
        lambda x: sum(label_encoder_week.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.week.tolist())))

    encode_cate['hour'] = room['hour'].apply(
        lambda x: sum(label_encoder_hour.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.hour.tolist())))

    encode_cate['DoctorSerNum'] = room['DoctorSerNum'].apply(
        lambda x: sum(label_encoder_DoctorSerNum.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.DoctorSerNum.tolist())))

    encode_cate['TreatmentOrientation'] = room['TreatmentOrientation'].apply(
        lambda x: sum(label_encoder_TreatmentOrientation.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.TreatmentOrientation.tolist())))

    encode_cate['FractionNumber'] = room['FractionNumber'].apply(
        lambda x: sum(label_encoder_FractionNumber.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.FractionNumber.tolist())))

    encode_cate['UserName'] = room['UserName'].apply(
        lambda x: sum(label_encoder_UserName.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.UserName.tolist())))

    encode_cate['CourseId'] = room['CourseId'].apply(
        lambda x: sum(label_encoder_CourseId.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.CourseId.tolist())))

    encode_cate['ResourceSerNum'] = room['ResourceSerNum'].apply(
        lambda x: sum(label_encoder_ResourceSerNum.transform(np.array(str(x)).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.ResourceSerNum.tolist())))
    
    encode_cate['AliasExpressionSerNum'] = room['AliasExpressionSerNum'].apply(
        lambda x: sum(label_encoder_AliasExpressionSerNum.transform(np.array(x).reshape(-1,1))))
    train_x = np.hstack((train_x, np.vstack(encode_cate.AliasExpressionSerNum.tolist())))
    
    # 将数值变量和非数值变量进行合并
    train_num = room[feature_num]
    train_x = np.hstack((train_x, train_num))
    
#     # 需要满足序列长度的要求，因此对于短序列进行补零操作
#     zeros = np.zeros((APPT_LEN - train_x.shape[0], FEATURE_LEN))
#     train_x = np.vstack((zeros, train_x))
    
    return train_x

In [ ]:
FEATURE_LEN = 463

In [ ]:
# # 不指定时间戳长度
# model_sequence = Sequential()
# model_sequence.add(layers.LSTM(
#     batch_input_shape = (None , None, FEATURE_LEN),
#     output_dim = 128,
#     dropout=0.1,
#     recurrent_dropout=0.5,
#     return_sequences=False,
# ))

# # model_sequence.add(layers.LSTM(
# #     output_dim = 32,
# #     return_sequences = False
# # ))
# # # stateful = True 本次batch的参数返回到下一次的训练中

# model_sequence.add(layers.Dense(32))

# model_sequence.add(layers.Dense(1))

# adam = optimizers.Adam(lr=0.001)
# model_sequence.compile(
#     optimizer = adam,
#     loss = 'mae'
# )

# model_sequence.summary()

In [210]:
room = room_info_grouped.get_group(room_seq[3])
room, label_d, label_n = generate_room_seq(room)
train_x = generate_sample(room, feature_num = feature_num)
# # 'pre_ActualEndDate' 'Transfer_duration', 'Interval_number'
room[['ScheduledStartTime', 'ScheduledEndTime', 'ActualStartDate', 'ActualEndDate', 'Scheduled_duration', 'Actual_duration', 'pre_ActualEndDate', 'Transfer_duration', 'Interval_number']]

,ScheduledStartTime,ScheduledEndTime,ActualStartDate,ActualEndDate,Scheduled_duration,Actual_duration,pre_ActualEndDate,Transfer_duration,Interval_number
125504,2015-06-17 08:00:00,2015-06-17 08:15:00,2015-06-17 07:46:00,2015-06-17 08:03:00,15.0,17.0,2015-06-17 07:46:00,0,0
125384,2015-06-17 08:15:00,2015-06-17 08:30:00,2015-06-17 08:16:00,2015-06-17 08:30:00,15.0,14.0,2015-06-17 08:03:00,780,0
125576,2015-06-17 09:00:00,2015-06-17 09:15:00,2015-06-17 08:37:00,2015-06-17 08:58:00,15.0,21.0,2015-06-17 08:30:00,420,0
125466,2015-06-17 08:30:00,2015-06-17 08:45:00,2015-06-17 08:59:00,2015-06-17 09:17:00,15.0,18.0,2015-06-17 08:58:00,60,0
125650,2015-06-17 08:45:00,2015-06-17 09:00:00,2015-06-17 09:18:00,2015-06-17 09:31:00,15.0,13.0,2015-06-17 09:17:00,60,0
125563,2015-06-17 09:30:00,2015-06-17 09:45:00,2015-06-17 09:31:00,2015-06-17 09:45:00,15.0,14.0,2015-06-17 09:31:00,0,0
125706,2015-06-17 10:15:00,2015-06-17 10:30:00,2015-06-17 10:00:00,2015-06-17 10:17:00,15.0,17.0,2015-06-17 09:45:00,900,1
125667,2015-06-17 11:15:00,2015-06-17 11:30:00,2015-06-17 10:23:00,2015-06-17 10:49:00,15.0,26.0,2015-06-17 10:17:00,360,0
125469,2015-06-17 11:45:00,2015-06-17 12:15:00,2015-06-17 11:59:00,2015-06-17 12:24:00,30.0,25.0,2015-06-17 10:49:00,4200,4
125731,2015-06-17 11:15:00,2015-06-17 11:30:00,2015-06-17 12:26:00,2015-06-17 12:44:00,15.0,18.0,2015-06-17 12:24:00,120,0


In [214]:
train_result_batch = 0
room_seq = shuffle(room_seq)

for i in range(len(room_seq)):
    
    # 构造训练数据
    room = room_info_grouped.get_group(room_seq[i])
    room, label_d, label_n = generate_room_seq(room)
    train_x = generate_sample(room)
    
    train_x = np.array([train_x])
    train_y = np.array([[label_d]])
    train_result = model_sequence.train_on_batch(train_x, train_y)
    
#     print(label_d)
#     print(train_result)
    if (i+1)%50 == 0:
        print(f'\nBatch {(i+1)/50} {train_result_batch/50}')
        train_result_batch = 0
    else:
        train_result_batch = train_result_batch + train_result


KeyboardInterrupt: 